# Feedforward Neural Network built on Keras

Below is a program built on Keras to build a regression model. 

## Dependencies

In this work, we need the numpy and pandas for processing raw data, and we will use keras (Tensorflow backend) an scikit-learn for building and evaluating regression models.

In [1]:
import numpy as np
import pandas as pd
import keras.callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.


## More preprocessing

In this step, we put all the idle and wild data into one file for the purpose of training and testing a regression model later. We also get rid of timestamps because we are using the emg and x,y,z values (features) at one moment and predicting the x,y,z values (targets) at the next moment. We could add timestamps into features as well.

In [58]:
files = ['miller_idleprocessed', 'sam_wildprocessed', 'layton_idleprocessed', 'miller_wildprocessed', 'val_idleprocessed', 
         'layton_wildprocessed', 'sam_idleprocessed', 'val_wildprocessed']
frames = [pd.read_csv(f, header = None) for f in files]
dataframe = pd.concat(frames)   #Concatenate all preprocessed files into one file

In [131]:
len(dataframe)  # A quick check on how many rows there are

74130

In [113]:
df = dataframe           # Give the processed dataframe a new name to do more processing
df = df.drop(1, axis=1)  # Drop a timestamp. Comment this line to keep it.
df = df.drop(5, axis=1)  # Drop another timestamp. Comment this line to keep it.

def merge_rows(df, starting_row):
    '''
    Merge different rows together; Slicing a row with the next 5 rows together (values on sixth row are the target).
    '''
    new_df = df[starting_row:starting_row + 6].as_matrix()
    new_df = new_df.ravel() #Flatten 
    new_df = pd.DataFrame(new_df).transpose() # Read in the flattened (raveled) np array into pandas and transpose it
    return new_df

% time merged_df = [merge_rows(df, i) for i in range(len(dataframe) - 6)]   # Let's see how much time it takes
# Concatnate the merged rows in the list together. (Without Concatnating them, they are just individual np arrays.)
concat_df = pd.concat(merged_df)  
concat_df


        0      2       3      4
0  4072.0 -0.328  0.2265 -0.949
1  4043.0 -0.316  0.2148 -0.937
2  4042.0 -0.335  0.2265 -0.945
3  4056.0 -0.324  0.2382 -0.937
4  3994.0 -0.335  0.2421 -0.957
5  4036.0 -0.328  0.2460 -0.953
6  4026.0 -0.328  0.2421 -0.949
7  4028.0 -0.324  0.2578 -0.945
8  4049.0 -0.312  0.2539 -0.957
9  4033.0 -0.304  0.2421 -0.929
CPU times: user 27.3 s, sys: 374 ms, total: 27.7 s
Wall time: 27.9 s


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,4072.0,-0.328,0.2265,-0.9490,4043.0,-0.316,0.2148,-0.9370,4042.0,-0.335,...,0.2382,-0.9370,3994.0,-0.335,0.2421,-0.9570,4036.0,-0.328,0.2460,-0.9530
0,4043.0,-0.316,0.2148,-0.9370,4042.0,-0.335,0.2265,-0.9450,4056.0,-0.324,...,0.2421,-0.9570,4036.0,-0.328,0.2460,-0.9530,4026.0,-0.328,0.2421,-0.9490
0,4042.0,-0.335,0.2265,-0.9450,4056.0,-0.324,0.2382,-0.9370,3994.0,-0.335,...,0.2460,-0.9530,4026.0,-0.328,0.2421,-0.9490,4028.0,-0.324,0.2578,-0.9450
0,4056.0,-0.324,0.2382,-0.9370,3994.0,-0.335,0.2421,-0.9570,4036.0,-0.328,...,0.2421,-0.9490,4028.0,-0.324,0.2578,-0.9450,4049.0,-0.312,0.2539,-0.9570
0,3994.0,-0.335,0.2421,-0.9570,4036.0,-0.328,0.2460,-0.9530,4026.0,-0.328,...,0.2578,-0.9450,4049.0,-0.312,0.2539,-0.9570,4033.0,-0.304,0.2421,-0.9290
0,4036.0,-0.328,0.2460,-0.9530,4026.0,-0.328,0.2421,-0.9490,4028.0,-0.324,...,0.2539,-0.9570,4033.0,-0.304,0.2421,-0.9290,4045.0,-0.312,0.2500,-0.9530
0,4026.0,-0.328,0.2421,-0.9490,4028.0,-0.324,0.2578,-0.9450,4049.0,-0.312,...,0.2421,-0.9290,4045.0,-0.312,0.2500,-0.9530,4043.0,-0.320,0.2500,-0.9570
0,4028.0,-0.324,0.2578,-0.9450,4049.0,-0.312,0.2539,-0.9570,4033.0,-0.304,...,0.2500,-0.9530,4043.0,-0.320,0.2500,-0.9570,4055.0,-0.308,0.2421,-0.9450
0,4049.0,-0.312,0.2539,-0.9570,4033.0,-0.304,0.2421,-0.9290,4045.0,-0.312,...,0.2500,-0.9570,4055.0,-0.308,0.2421,-0.9450,4093.0,-0.312,0.2460,-0.9570
0,4033.0,-0.304,0.2421,-0.9290,4045.0,-0.312,0.2500,-0.9530,4043.0,-0.320,...,0.2421,-0.9450,4093.0,-0.312,0.2460,-0.9570,4097.0,-0.300,0.2421,-0.9640


In [129]:
concat_df = concat_df.reset_index(drop = True)  # Reset the index numbers to look better
concat_df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,4072.0,-0.328,0.2265,-0.9490,4043.0,-0.316,0.2148,-0.9370,4042.0,-0.335,...,0.2382,-0.9370,3994.0,-0.335,0.2421,-0.9570,4036.0,-0.328,0.2460,-0.9530
1,4043.0,-0.316,0.2148,-0.9370,4042.0,-0.335,0.2265,-0.9450,4056.0,-0.324,...,0.2421,-0.9570,4036.0,-0.328,0.2460,-0.9530,4026.0,-0.328,0.2421,-0.9490
2,4042.0,-0.335,0.2265,-0.9450,4056.0,-0.324,0.2382,-0.9370,3994.0,-0.335,...,0.2460,-0.9530,4026.0,-0.328,0.2421,-0.9490,4028.0,-0.324,0.2578,-0.9450
3,4056.0,-0.324,0.2382,-0.9370,3994.0,-0.335,0.2421,-0.9570,4036.0,-0.328,...,0.2421,-0.9490,4028.0,-0.324,0.2578,-0.9450,4049.0,-0.312,0.2539,-0.9570
4,3994.0,-0.335,0.2421,-0.9570,4036.0,-0.328,0.2460,-0.9530,4026.0,-0.328,...,0.2578,-0.9450,4049.0,-0.312,0.2539,-0.9570,4033.0,-0.304,0.2421,-0.9290
5,4036.0,-0.328,0.2460,-0.9530,4026.0,-0.328,0.2421,-0.9490,4028.0,-0.324,...,0.2539,-0.9570,4033.0,-0.304,0.2421,-0.9290,4045.0,-0.312,0.2500,-0.9530
6,4026.0,-0.328,0.2421,-0.9490,4028.0,-0.324,0.2578,-0.9450,4049.0,-0.312,...,0.2421,-0.9290,4045.0,-0.312,0.2500,-0.9530,4043.0,-0.320,0.2500,-0.9570
7,4028.0,-0.324,0.2578,-0.9450,4049.0,-0.312,0.2539,-0.9570,4033.0,-0.304,...,0.2500,-0.9530,4043.0,-0.320,0.2500,-0.9570,4055.0,-0.308,0.2421,-0.9450
8,4049.0,-0.312,0.2539,-0.9570,4033.0,-0.304,0.2421,-0.9290,4045.0,-0.312,...,0.2500,-0.9570,4055.0,-0.308,0.2421,-0.9450,4093.0,-0.312,0.2460,-0.9570
9,4033.0,-0.304,0.2421,-0.9290,4045.0,-0.312,0.2500,-0.9530,4043.0,-0.320,...,0.2421,-0.9450,4093.0,-0.312,0.2460,-0.9570,4097.0,-0.300,0.2421,-0.9640


In [145]:

X = concat_df[concat_df.columns[0:20]]  # X is the features set
Y = concat_df[concat_df.columns[21:]]   # Y is the targets set
X = X.values    #Convert to np arrays
Y = Y.values    #Convert to np arrays

print (X, "\n", Y)  # See if it looks good

[[  4.07200000e+03  -3.28000000e-01   2.26500000e-01 ...,  -3.35000000e-01
    2.42100000e-01  -9.57000000e-01]
 [  4.04300000e+03  -3.16000000e-01   2.14800000e-01 ...,  -3.28000000e-01
    2.46000000e-01  -9.53000000e-01]
 [  4.04200000e+03  -3.35000000e-01   2.26500000e-01 ...,  -3.28000000e-01
    2.42100000e-01  -9.49000000e-01]
 ..., 
 [  3.46800000e+03  -5.40000000e-02  -7.50000000e-01 ...,  -3.50000000e-02
   -7.10000000e-01   6.21000000e-01]
 [  3.48900000e+03  -5.80000000e-02  -7.18000000e-01 ...,  -5.40000000e-02
   -7.07000000e-01   6.25000000e-01]
 [  3.49400000e+03  -7.00000000e-02  -7.30000000e-01 ...,  -4.20000000e-02
   -7.14000000e-01   6.17100000e-01]] 
 [[-0.328   0.246  -0.953 ]
 [-0.328   0.2421 -0.949 ]
 [-0.324   0.2578 -0.945 ]
 ..., 
 [-0.054  -0.707   0.625 ]
 [-0.042  -0.714   0.6171]
 [-0.058  -0.726   0.6171]]


## Build the model by using Keras

* In this part, I benefited a lot from this post: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/.
* I put 16 hidden layers and use relu as the activation layer. (It turns out that if I put the number of hidden layers under 10, it doesn't really work.) I use mean_squared_error as the loss function. And I use the adam optimizer. (Adam is better than RMSprop based on my experiment. In one experiment I did with RMSprop, the r2 score is only around 0.89, which is around 9 percent less than when using adam. RMSprop optimizer also learns slower.)
* The official Keras document on Sequential model can be found here: https://keras.io/getting-started/sequential-model-guide/

In [146]:
def baseline_model():
    model = Sequential()
    model.add(Dense(32, input_dim = 20, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(3, kernel_initializer = 'normal'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    return model

## Testing performance of the model

* A large part of the code here is from Layton. It uses the modules in scikit-learn. The normal scoring of my model, I believe, is mean sqaured error. I changed into r2 in order to have better comparison metrics. It we get rid of  *scoring = 'r2'*, then the printed resulted reg.score will be mean squared error. 
* Here are a couple helpful links from scikit-learn documents on how to set parameters for cross validation here. 
    - http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    - http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [147]:

# from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

from sklearn.model_selection import GridSearchCV

#hyperparameters to search through with CV
parameters = {
    'epochs' : [100],
    'batch_size' : [64,128], # The larger the batch size, the faster the model will run.
    'verbose' : [2] # for debugging purpose. It will significantly lower the running speed. To speed up, change it to 0.
}

#searching through all combinations of parameters to find best (ranked by CV)
reg = GridSearchCV(KerasRegressor(build_fn=baseline_model), parameters, scoring = 'r2')
reg.fit(X_train, y_train)

#scoring against test set to see how good our model is
print(reg.score(X_test, y_test))

Epoch 1/100
 - 2s - loss: 198.5847
Epoch 2/100
 - 1s - loss: 3.6170
Epoch 3/100
 - 1s - loss: 2.4993
Epoch 4/100
 - 1s - loss: 1.9475
Epoch 5/100
 - 1s - loss: 1.2819
Epoch 6/100
 - 1s - loss: 0.9986
Epoch 7/100
 - 1s - loss: 0.6826
Epoch 8/100
 - 1s - loss: 0.4956
Epoch 9/100
 - 1s - loss: 0.7776
Epoch 10/100
 - 1s - loss: 0.7714
Epoch 11/100
 - 2s - loss: 1.2794
Epoch 12/100
 - 1s - loss: 0.3443
Epoch 13/100
 - 1s - loss: 1.5602
Epoch 14/100
 - 1s - loss: 0.4402
Epoch 15/100
 - 1s - loss: 0.4736
Epoch 16/100
 - 1s - loss: 0.6440
Epoch 17/100
 - 1s - loss: 0.6276
Epoch 18/100
 - 1s - loss: 0.8338
Epoch 19/100
 - 1s - loss: 0.1870
Epoch 20/100
 - 1s - loss: 0.3697
Epoch 21/100
 - 1s - loss: 0.6945
Epoch 22/100
 - 1s - loss: 0.2689
Epoch 23/100
 - 1s - loss: 0.3989
Epoch 24/100
 - 1s - loss: 0.6362
Epoch 25/100
 - 1s - loss: 0.5586
Epoch 26/100
 - 1s - loss: 0.2443
Epoch 27/100
 - 1s - loss: 0.1428
Epoch 28/100
 - 1s - loss: 0.2839
Epoch 29/100
 - 1s - loss: 0.1425
Epoch 30/100
 - 1s - 

## Let's print some results and see the performance!

Code here is from Layton too.

In [ ]:
print('data:')
print(X_test[:10])
print('\npredictions:')
print(reg.predict(X_test)[:10])
print('\ntargets:')
print(y_test[:10])